In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-12-23"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
17043,2024-12-23,Brasil Nbb,19:30,Minas,São Paulo,1.13,4.85,160.5,1.88,1.78,-12.5,2.01,1.68,jNzk8mBG,0.884956,0.206186,0.531915,0.561798,0.091141,0.4,0.6,NaN,NaN,121.184,51.483487,0.424837,102.124,12.422644,0.121643,98.024,128.772,103.20,113.60,0.0,0.0,0.0,0.0,0.879745,0.038640,0.126474,0.62,0.124,1.048387,0.823976,0.8,-0.023976,0.03,0.006,641.666667,0.550106,0.6,0.049894
17044,2024-12-23,Brasil Nbb,19:30,Paulistano,Flamengo,2.41,1.47,153.5,1.88,1.78,3.5,1.95,1.72,OzZd6RuT,0.414938,0.680272,0.531915,0.561798,0.095210,0.6,0.6,NaN,NaN,165.548,96.241766,0.581353,104.448,15.722415,0.150529,100.408,111.698,91.02,130.90,0.0,0.0,0.0,0.0,0.342619,0.038640,0.088629,-1.33,-0.266,-5.300752,0.690709,0.4,-0.290709,-0.27,-0.054,-8.703704,0.767027,0.8,0.032973
17045,2024-12-23,Brasil Nbb,19:30,União Corinthians,Pinheiros,1.68,2.01,155.5,1.88,1.78,-3.5,2.03,1.67,6exM053j,0.595238,0.497512,0.531915,0.561798,0.092751,0.6,0.6,NaN,NaN,184.242,33.894733,0.183969,180.070,69.173421,0.384147,168.624,243.738,205.92,177.10,0.0,0.0,0.0,0.0,0.126474,0.038640,0.137599,-1.38,-0.276,-2.463768,0.517274,0.5,-0.017274,4.83,0.966,1.045549,0.365695,0.6,0.234305
17046,2024-12-23,Brasil Nbb,20:00,Botafogo,Franca,6.03,1.08,160.5,1.80,1.86,12.5,2.01,1.68,Y1U7O8ec,0.165837,0.925926,0.555556,0.537634,0.091763,0.6,0.6,NaN,NaN,308.768,117.158162,0.379438,101.926,15.105568,0.148201,210.144,107.402,303.18,124.20,1.0,0.0,0.0,0.0,0.984579,0.023184,0.126474,-1.74,-0.348,-14.454023,0.400405,0.2,-0.200405,-1.02,-0.204,-0.392157,0.664893,0.6,-0.064893
17047,2024-12-23,Brasil Nbb,20:00,Caxias do Sul,Corinthians Paulista,2.41,1.47,156.5,1.82,1.84,3.5,1.95,1.72,2sMOKQfM,0.414938,0.680272,0.549451,0.543478,0.095210,0.4,0.2,NaN,NaN,282.174,109.382749,0.387643,120.048,27.654286,0.230360,174.418,213.350,311.60,102.24,1.0,0.0,0.0,0.0,0.342619,0.007728,0.088629,-3.61,-0.722,-1.952909,0.443466,0.2,-0.243466,6.49,1.298,0.362096,0.491188,0.8,0.308812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17102,2024-12-23,Eua Ncaa,22:00,Oakland,Oregon State,6.00,1.12,130.5,1.80,1.86,9.5,2.03,1.67,fPxzGBJD,0.166667,0.892857,0.555556,0.537634,0.059524,0.6,0.6,NaN,NaN,256.390,259.004041,1.010196,123.928,62.587403,0.505030,113.918,344.812,118.80,182.39,0.0,0.0,0.0,0.0,0.969292,0.023184,0.137599,1.09,0.218,22.935780,0.718019,0.9,0.181981,-5.00,-1.000,-0.120000,0.000000,0.0,0.000000
17103,2024-12-23,França Lnb,15:00,Paris,Limoges,1.15,5.50,168.5,1.86,1.90,-12.5,1.99,1.81,QLvS2bu0,0.869565,0.181818,0.537634,0.526316,0.051383,0.8,0.2,NaN,NaN,154.956,55.315739,0.356977,196.916,126.913449,0.644506,141.084,316.396,186.48,191.62,0.0,0.0,0.0,0.0,0.925087,0.015045,0.066989,0.13,0.026,5.769231,0.750329,0.8,0.049671,-5.00,-1.000,-4.500000,0.323530,0.1,-0.223530
17104,2024-12-23,Israel Superliga,13:30,Ironi Kiryat Ata,Galil Elyon,1.99,1.77,160.5,1.81,1.93,1.5,1.86,1.88,2cWVug0c,0.502513,0.564972,0.552486,0.518135,0.067484,0.2,0.4,NaN,NaN,277.180,305.036589,1.100500,276.908,197.866207,0.714556,214.020,401.142,82.56,576.00,0.0,1.0,0.0,0.0,0.082747,0.045376,0.007563,6.84,1.368,0.723684,0.516703,0.6,0.083297,-0.98,-0.196,-3.928571,0.331043,0.4,0.068957
17105,2024-12-23,Polônia 1. Liga,14:00,MKKS Koszalin,Polonia Warszawa,1.97,1.71,165.5,1.84,1.82,-3.5,2.01,1.68,MerAcFuA,0.507614,0.5847

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:00,Eua Nba,Charlotte Hornets,Houston Rockets,215.5,1.90,0.6111,Over
1,21:00,Eua Nba,Orlando Magic,Boston Celtics,217.5,1.84,0.8111,Over
2,21:00,Eua Nba,Philadelphia 76ers,San Antonio Spurs,216.5,1.84,0.9853,Over
3,21:30,Eua Nba,Miami Heat,Brooklyn Nets,214.5,1.84,0.8922,Over
4,22:00,Eua Nba,Chicago Bulls,Milwaukee Bucks,232.5,1.84,0.9831,Over
5,22:00,Eua Nba,Memphis Grizzlies,Los Angeles Clippers,230.5,1.93,0.8456,Over
6,22:00,Eua Nba,Oklahoma City Thunder,Washington Wizards,221.5,1.84,0.9826,Over
7,22:30,Eua Nba,Dallas Mavericks,Portland Trail Blazers,228.5,1.81,0.8864,Over
8,14:00,Croácia Copa Da Croácia,Split,GKK Sibenka,161.5,1.88,0.9999,Over
